In [1]:
import pandas as pd
import numpy as np

ds = pd.read_csv('data\Retail 2010-11.csv')
ds.shape

(541910, 8)

In [2]:
ds.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [3]:
ds.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541910 entries, 0 to 541909
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype  
---  ------       --------------   -----  
 0   Invoice      541910 non-null  object 
 1   StockCode    541910 non-null  object 
 2   Description  540456 non-null  object 
 3   Quantity     541910 non-null  int64  
 4   InvoiceDate  541910 non-null  object 
 5   Price        541910 non-null  float64
 6   Customer ID  406830 non-null  float64
 7   Country      541910 non-null  object 
dtypes: float64(2), int64(1), object(5)
memory usage: 33.1+ MB


In [4]:
ds.isnull().sum()

Invoice             0
StockCode           0
Description      1454
Quantity            0
InvoiceDate         0
Price               0
Customer ID    135080
Country             0
dtype: int64

In [5]:
ds['Customer ID'] = ds['Customer ID'].fillna(0)
ds['Customer ID'] = ds['Customer ID'].astype(int)
ds.rename(columns={'Customer ID' : "CustomerID"}, inplace=True)

In [6]:
ds.dropna(subset=['Description'], inplace=True)

In [7]:
# Transactions with invoices starting with 'C' are cancellations. Create a new column to flag these cancellations
ds['Cancelled'] = ds['Invoice'].astype(str).apply(lambda x: 1 if x.startswith('C') else 0)

In [8]:
import re
ds['Invoice'] = ds['Invoice'].apply(lambda x: re.sub('\D', '', x))
ds['Invoice'] = pd.to_numeric(ds['Invoice'])
ds['StockCode'] = ds['StockCode'].apply(lambda x: re.sub('\D', '', x))
ds.drop(ds[ds.StockCode == ''].index, inplace=True)
ds['StockCode'] = ds['StockCode'].astype(int)

In [9]:
ds['InvoiceDate'] = ds['InvoiceDate'].astype('datetime64[ns]')
ds['InvoiceDate'] = ds['InvoiceDate'].dt.strftime('%Y-%m-%d')

ds['Numerical_Date'] = pd.to_datetime(ds['InvoiceDate']).dt.dayofyear

In [10]:
#ds = pd.get_dummies(ds, columns=["Country"])
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#ds['Description_encoded'] = le.fit_transform(ds['Description'])
ds['Country_encoded'] = le.fit_transform(ds['Country'])

In [11]:
ds['Quantity'] = ds['Quantity'].abs()
#ds['Price'] = ds['Price'].abs()

In [12]:
# Create a new column for the total amount of each transaction
ds['Total'] = ds['Quantity'] * ds['Price']

In [13]:
# make one column ordered quantity and put value from quantity column to ordered quantity when Cancelled column has 0 value
ds['OrderedQuantity'] = np.where(ds['Cancelled'] == 0, ds['Quantity'], 0)

In [14]:
# make one column cancelled quantity and put value from quantity column to ordered quantity when Cancelled column has 1 value
ds['CancelledQuantity'] = np.where(ds['Cancelled'] == 1, ds['Quantity'], 0)

In [15]:
ds.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,CustomerID,Country,Cancelled,Numerical_Date,Country_encoded,Total,OrderedQuantity,CancelledQuantity
0,536365,85123,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-01-12,2.55,17850,United Kingdom,0,12,36,15.30,6,0
1,536365,71053,WHITE METAL LANTERN,6,2010-01-12,3.39,17850,United Kingdom,0,12,36,20.34,6,0
2,536365,84406,CREAM CUPID HEARTS COAT HANGER,8,2010-01-12,2.75,17850,United Kingdom,0,12,36,22.00,8,0
3,536365,84029,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-01-12,3.39,17850,United Kingdom,0,12,36,20.34,6,0
4,536365,84029,RED WOOLLY HOTTIE WHITE HEART.,6,2010-01-12,3.39,17850,United Kingdom,0,12,36,20.34,6,0


Data Modelling

In [16]:
#Training data preparation
from sklearn.model_selection import train_test_split

X = ds.drop(['Quantity', 'OrderedQuantity','CancelledQuantity','Description','InvoiceDate','Country'], axis=1)
y = ds['OrderedQuantity']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X

,Invoice,StockCode,Price,CustomerID,Cancelled,Numerical_Date,Country_encoded,Total
0,536365,85123,2.55,17850,0,12,36,15.30
1,536365,71053,3.39,17850,0,12,36,20.34
2,536365,84406,2.75,17850,0,12,36,22.00
3,536365,84029,3.39,17850,0,12,36,20.34
4,536365,84029,3.39,17850,0,12,36,20.34
...,...,...,...,...,...,...,...,...
541904,581587,22613,0.85,12680,0,255,13,10.20
541905,581587,22899,2.10,12680,0,255,13,12.60
541906,581587,23254,4.15,12680,0,255,13,16.60
541907,581587,23255,4.15,12680,0,255,13,16.60


In [17]:
#Training the model
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeClassifier

#Random Forest model
RFmodel = RandomForestRegressor()
RFmodel.fit(X_train, y_train)

#Decision Tree model
DTmodel = DecisionTreeClassifier()
DTmodel.fit(X_train, y_train)

DecisionTreeClassifier()

In [18]:
#Saving model using Joblib
import joblib
joblib.dump(RFmodel, 'models/RFmodel.joblib')
joblib.dump(DTmodel, 'models/DTmodel.joblib')

['models/DTmodel.joblib']

In [19]:
from sklearn.metrics import r2_score

# Predictions
y_pred = DTmodel.predict(X)

# Compute R² Score
r2 = r2_score(y, y_pred)
print(f'R² Score of Decision Tree Model: {r2:.2f}')

y_pred2 = RFmodel.predict(X)

r2 = r2_score(y, y_pred2)
print(f'R² Score of Random Forest Model: {r2:.2f}')


R² Score of Decision Tree Model: 1.00
R² Score of Random Forest Model: 0.90


In [20]:
#Saving CSV file
ds.to_csv('data\data_cleaned.csv', index=False)

In [ ]:
ndata = {'Invoice': [581588], 
        'StockCode': [22137], 
        'Price': [4], 
        'Price': [6.95], 
        'CustomerID': [12678], 
        'Cancelled': [0],
        'Numerical_Date': [343],
        'Country_encoded': [13], 
        'Total': [19.8]}
new_data = pd.DataFrame(data=ndata)
#(581587	22138	4.95	12680	0	255	13	14.85)
new_data

,Invoice,StockCode,Price,CustomerID,Cancelled,Numerical_Date,Country_encoded,Total
0,581588,22137,6.95,12678,0,343,0,19.8


In [28]:
n_pred = DTmodel.predict(new_data)
n_pred

array([3], dtype=int64)

In [21]:
# #Filling null values
# ds['Description'] = ds['Description'].fillna('No description Available')
# ds['CustomerID'] = ds['CustomerID'].fillna(0)

In [22]:
# #Formatting InvoiceDate
# ds["InvoiceDate"] = pd.to_datetime(ds["InvoiceDate"])
# ds["InvoiceDate"] = ds['InvoiceDate'].dt.strftime('%m-%d-%Y')
# ds['InvoiceDate'] = pd.to_numeric(pd.to_datetime(ds['InvoiceDate']))

In [23]:
# #Formatting columns to correct data types
# ds['InvoiceNo'] = ds['InvoiceNo'].astype('object')
# ds['StockCode'] = ds['StockCode'].astype('str')
# ds['CustomerID'] = ds['CustomerID'].astype('int64')
# ds['Quantity'] = ds['Quantity'].astype('int')


In [24]:
# #Removing charater in StockCode
# ds = ds[ds.Quantity > 0]
# ds['StockCode'] = ds['StockCode'].str.extract('(\d+)')
# ds = ds.dropna(subset=['StockCode'])
# ds['StockCode'] = ds['StockCode'].astype('int64')

# # ds['StockCode'] = ds['StockCode'].str.extract('(\d+)')
# # ds = ds.dropna(subset=['StockCode'])
# # ds['StockCode'] = ds['StockCode'].astype('int64')

In [25]:
# #Adding a new column for total price
# ds["TotalPrice"] = ds["Quantity"] * ds["UnitPrice"]
# ds.head()